In [2]:
import pickle
import fastf1 as ff1
import pandas as pd
import numpy as np

from fastf1.core import Session
from typing import List, Iterable

import f1_pitstop_advisor.idea_one.data_processing_utils as utils

from sklearn.pipeline import make_pipeline

In [3]:
with open("ig_sessions.pickle", "rb") as file:
    sessions = pickle.load(file)

In [4]:
# Organize sessions by circuit
data_by_circuit = {}
for session in sessions:
    circuit_key =  session.session_info["Meeting"]["Circuit"]["Key"]
    if circuit_key not in data_by_circuit.keys():
        data_by_circuit[circuit_key] = []
    data_by_circuit[circuit_key].append(session)

In [ ]:
# Get lap data combined with weather data
data = utils.get_lap_data_with_weather(session).convert_dtypes()

# Add a column representing lap time in seconds
data["LapTimeSeconds"] = data["LapTime"].apply(lambda t: t.total_seconds())

In [ ]:
# Create a DataFrame with drivers as rows and mean/standard deviation of their lap times for columns
driver_codes = data["Driver"].unique()
driver_df = pd.DataFrame(index=driver_codes)
driver_df["MeanTime"] = None
driver_df["StandardDeviation"] = None

for driver in driver_codes:
    driver_times = data[data["Driver"] == driver]["LapTimeSeconds"]
    driver_df.loc[driver, "MeanTime"] = driver_times.mean()
    driver_df.loc[driver, "StandardDeviation"] = driver_times.std()


In [ ]:
# Calculate Z-score for each lap. This score will tell us how good or bad a lap is 
# relative to the rest of the same driver's laps. Lower is better.

data["LapTimeZScore"] = None
average_stdev = driver_df["StandardDeviation"].mean()
for idx in data.index:
    subset = data.loc[idx, ["LapTimeSeconds", "Driver"]]
    # Lap time for the current lap
    lap_time = subset["LapTimeSeconds"]
    driver = subset["Driver"]
    # Mean lap time for the driver who took the current lap
    mean_time = driver_df.loc[driver, "MeanTime"]
    # Add Z-score
    data.loc[idx, "LapTimeZScore"] = (lap_time - mean_time) / average_stdev


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,IsAccurate,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,LapTimeSeconds,LapTimeZScore
0,0 days 00:59:18.340000,NOR,4,0 days 00:01:30.120000,1,1,NaT,NaT,NaT,0 days 00:00:31.396000,...,False,21.7,58,987.2,False,31.9,254,2.5,90.120,2.640774
1,0 days 01:00:42.304000,NOR,4,0 days 00:01:23.964000,2,1,NaT,NaT,0 days 00:00:30.209000,0 days 00:00:30.346000,...,True,21.8,57,987.3,False,32.3,284,2.9,83.964,0.702725
2,0 days 01:02:05.527000,NOR,4,0 days 00:01:23.223000,3,1,NaT,NaT,0 days 00:00:29.792000,0 days 00:00:30.097000,...,True,21.8,58,987.2,False,31.7,286,3.2,83.223,0.469441
3,0 days 01:03:28.404000,NOR,4,0 days 00:01:22.877000,4,1,NaT,NaT,0 days 00:00:30.125000,0 days 00:00:29.665000,...,True,21.8,58,987.3,False,31.5,263,2.9,82.877,0.360513
4,0 days 01:04:51.043000,NOR,4,0 days 00:01:22.639000,5,1,NaT,NaT,0 days 00:00:29.930000,0 days 00:00:29.672000,...,True,21.8,59,987.2,False,31.4,277,3.1,82.639,0.285585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,0 days 01:59:27.312000,BEA,87,0 days 00:01:22.527000,44,2,NaT,NaT,0 days 00:00:29.675000,0 days 00:00:29.516000,...,True,22.2,56,987.1,False,31.5,242,2.0,82.527,-0.514558
1364,0 days 02:00:49.842000,BEA,87,0 days 00:01:22.530000,45,2,NaT,NaT,0 days 00:00:29.726000,0 days 00:00:29.507000,...,True,22.2,55,987.1,False,31.3,293,1.7,82.530,-0.513614
1365,0 days 02:02:13.856000,BEA,87,0 days 00:01:24.014000,46,2,NaT,NaT,0 days 00:00:30.190000,0 days 00:00:30.343000,...,True,22.1,55,987.0,False,31.2,268,1.8,84.014,-0.046417
1366,0 days 02:03:38.249000,BEA,87,0 days 00:01:24.393000,47,2,NaT,NaT,0 days 00:00:30.117000,0 days 00:00:30.648000,...,True,22.1,56,987.1,False,30.9,266,2.1,84.393,0.072901
